# Extreme Value Experimentation

In [1]:
import feedparser
import pprint
import datetime as dt
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
import requests
import time
import numpy as np
from pyextremes import get_extremes
from pyextremes.plotting import plot_extremes
import matplotlib.pyplot as plt
import sqlite3

# Display all columns
pd.set_option('display.max_columns', None)

# Display numbers in full (no scientific notation)
pd.set_option('display.float_format', '{:.4f}'.format)

Potential Python packages include:  
- ``pyextremes``
- ``scipy.stats``

## Database Connections/Custom Python Packages

In [2]:
# imports
import pandas_market_calendars as mcal
import wrds

In [3]:
import sys
sys.path.append("/Users/audreymcmillion/Documents/acm-thesis") 
from ev_scoring import ExtremeValueScoring
ev = ExtremeValueScoring(wrds_username='audreymcmillion')

Loading library list...
Done


In [4]:
db = ev.wrds_db
conn = ev.sqlite_conn

In [5]:
from market_utils import MarketUtilities
mkt_utils = MarketUtilities(wrds_username='audreymcmillion', wrds_db = db, sqlite_conn = conn)

## Read in Anomaly Datasets

In [ ]:
low_anom = pd.read_csv("test_data/low_anomalies.csv")
high_anom = pd.read_csv("test_data/high_anomalies.csv")

In [ ]:
high_anom.sort_values("current_date", ascending=False)

We will look into one particular anomalous symbol day: PALI on 2023-09-06.

In [ ]:
luld_data = pd.read_csv("test_data/halt_records.csv")

## Test a single (symbol, date)

In [ ]:
luld_data[(luld_data.symbol == 'PALI') & (luld_data.halt_date == '09/06/2023')]

In [ ]:
pali_trades = db.raw_sql("""
    with rw_trades as (
    	select *,
    	       date_trunc('second', time_m) as trunc_time
    	from taqm_2023.ctm_2023 c 
    	where c.date between date('2023-08-15') and date('2023-09-27')
    	and c.sym_root = 'PALI'
    	and time_m >= '09:30:00'
    	and time_m <= '16:00:00'
    	and tr_scond !~ '[OPQ65]'
    ),

    aggregated as (
        select r.date, 
               r.sym_root, 
               r.trunc_time, 
               avg(r.price) as avg_price,
               max(r.price) as max_price,
               min(r.price) as min_price,
               sum(r.size) as volume, 
               max(r.tr_seqnum) as tr_seqnum
        from rw_trades r 
        group by r.date, r.sym_root, r.trunc_time
    )

    select a.*,
           (a.avg_price - lag(a.avg_price) over (
                        partition by a.date 
                        order by a.date, a.trunc_time
                        )) AS avg_price_diff,
            EXTRACT(EPOCH FROM (a.trunc_time - LAG(a.trunc_time) OVER (
                        PARTITION BY a.date 
                        ORDER BY a.date, a.trunc_time))) AS time_delta
    from aggregated a
    order by a.date, a.trunc_time
    """)

In [ ]:
pali_trades['trunc_time'] = (pd.to_datetime('00:00:00') + pali_trades['trunc_time']).dt.time

In [ ]:
pali_trades['datetime'] = pd.to_datetime(pali_trades['date'].astype(str) + ' ' + pali_trades['trunc_time'].astype(str))

In [ ]:
pali_trades = pali_trades.set_index("datetime")

In [ ]:
pali_trades[pali_trades.date == dt.date(2023, 9, 6)]

In [ ]:
pali_trades[(pali_trades.date == dt.date(2023, 9, 6)) & (pali_trades.trunc_time <= dt.time(9, 38, 34))][:-50]

In [ ]:
pali_trades[pali_trades.date == dt.date(2023, 9, 6)].dropna().volume.plot()

In [ ]:
pali_trades[pali_trades.date == dt.date(2023, 9, 6)].dropna().avg_price_diff.plot()
plt.show()

In [ ]:
pali_trades.dropna().avg_price_diff.plot()
plt.show()

In [ ]:
# using the block maxima (BM) method
extremes = get_extremes(pali_trades.dropna().avg_price_diff, "BM", block_size="1H", errors="ignore")
plot_extremes(
    ts=pali_trades.dropna().avg_price_diff,
    extremes=extremes,
    extremes_method="BM",
    extremes_type="high",
    block_size="1H",
)
plt.show()

In [ ]:
extremes

In [ ]:
from pyextremes import get_extremes, get_return_periods

return_periods = get_return_periods(
    ts=pali_trades.dropna().avg_price_diff,
    extremes=extremes,
    extremes_method="BM",
    extremes_type="high",
    block_size="1H",
    return_period_size="1H",
    plotting_position="weibull",
)
return_periods.sort_values("return period", ascending=False).head()

From this, could we use the exeedance probability or the return period to "score" price movements.

## Fit an extreme value model and extract parameters

In [ ]:
from pyextremes import EVA
model = EVA(pali_trades.dropna().avg_price_diff)

In [ ]:
model.get_extremes(method="BM", block_size="1H", errors="ignore")

In [ ]:
model.plot_extremes()

In [ ]:
# fit the model
model.fit_model()

In [ ]:
1 - model.model.cdf(pali_trades.dropna().avg_price_diff.max())

In [ ]:
model.plot_diagnostic(alpha=0.95, return_period_size="1H")

From the below QQ plot, we are able obtain a QQ plot with an $R^2$ value and p value.

**Testing for model fit**

In [ ]:
from pyextremes.tests import KolmogorovSmirnov
from scipy import stats

In [ ]:
kstest = KolmogorovSmirnov(
            extremes=model.extremes,
            distribution=model.distribution.name,
            fit_parameters=model.distribution.mle_parameters,
            significance_level=0.05,
        )

In [ ]:
kstest

**NOTE**: The critical value of the above is implemented incorrectly, using ``stats.ksone`` instead of ``stats.kstwo`` to obtain the p-value of the fit.

In [ ]:
kstest.pvalue

In [ ]:
kstest.test_statistic

In [ ]:
critical_value = stats.kstwo.ppf(1 - 0.05 , len(model.extremes))
critical_value

In [ ]:
from scipy import stats

stats.kstest(model.extremes, 
             model.distribution.name, 
             args=(tuple(model.distribution.mle_parameters.values())), 
             alternative="two-sided")

## Assigning Intraday Extreme Value Scores 

In [7]:
run_pdata = True

In [8]:
anom_df = pd.read_sql("""
           with anom_details as (
            	select distinct bar.ticker, 
                                bar.current_date, 
                                bar.before_date, 
                                bar.after_date, 
                                bar.before_avg_dlynumtrd, 
                                bar.dlynumtrd, 
                                bar.after_avg_dlynumtrd
            	from before_after_results bar
            	where bar.open_pr is not null
            	and bar.close_pr is not null
            	and bar.dlyhigh is not null
            	and bar.dlylow is not null
            	and bar.dlynumtrd is not null
            )

            select a.ticker, 
                   a.current_date, 
                   a.before_date, 
                   a.after_date,
                   (a.before_avg_dlynumtrd + a.after_avg_dlynumtrd)/2.0 as avg_dlynumtrd,
                   a.dlynumtrd
            from anom_details a
    """, conn)

In [11]:
anom_df["current_date"].max()

'2023-12-06'

Now, score extreme values from this dataset:

In [12]:
anom_df = anom_df.drop_duplicates(subset=["ticker", "current_date"]).reset_index(drop=True)

In [13]:
anom_df

,ticker,current_date,before_date,after_date,avg_dlynumtrd,dlynumtrd
0,ENDP,2013-05-10,2013-04-19,2013-06-03,12971.4667,18771.0000
1,ZNGA,2013-06-03,2013-05-10,2013-06-24,28317.4333,73590.0000
2,MYGN,2013-06-13,2013-05-22,2013-07-05,12745.1333,82062.0000
3,ALXN,2013-07-12,2013-06-20,2013-08-02,15007.2667,63272.0000
4,HMNY,2013-08-05,2013-07-15,2013-08-26,135.0000,16.0000
...,...,...,...,...,...,...
17462,IXHL,2023-12-06,2023-11-30,2023-12-28,3848.4750,4007.0000
17463,WBUY,2023-12-06,2023-11-14,2023-12-28,4713.7000,61594.0000
17464,EFHT,2023-12-06,2023-11-14,2023-12-28,1763.9000,568.0000
17465,CHSN,2023-12-06,2023-11-14,2023-12-28,6596.4333,23129.0000


In [ ]:
# anom_df

In [ ]:
# anom_df

In [ ]:
already_processed = pd.read_csv("processed_data.csv").drop_duplicates(subset=["ticker", "current_date"]).reset_index(drop=True)

In [ ]:
# already_processed

In [ ]:
# Perform a left merge with indicator
merged_df = anom_df.merge(already_processed[['ticker', 'current_date']], on=['ticker', 'current_date'], how='left', indicator=True)

# Filter rows where the indicator is 'left_only'
filtered_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

In [ ]:
# filtered_df = anom_df

In [ ]:
filtered_df

In [ ]:
if run_pdata:
    # chunk the DataFrame into 100-row chunks and process each chunk
    chunk_size = 100
    output_file = 'processed_data.csv'
    
    for start in tqdm(range(0, len(filtered_df), chunk_size)):
        chunk = filtered_df[start:start + chunk_size]
        processed_chunk = ev.process_intraday_data(chunk)
        
        # write to CSV
        # if start == 0:
            # Write the header in the first iteration
        #    processed_chunk.to_csv(output_file, mode='w', index=False)
        # else: 
        processed_chunk.to_csv(output_file, mode='a', index=False, header=False)

## Assigning Interday Extreme Value Scores

In [6]:
run_inter_pdata = True

In [7]:
inter_anom_df = pd.read_sql("""
           with anom_details as (
            	select distinct bar.ticker, 
                                bar.current_date, 
                                bar.before_date, 
                                bar.after_date, 
                                bar.before_avg_dlynumtrd, 
                                bar.dlynumtrd, 
                                bar.after_avg_dlynumtrd
            	from before_after_results bar
            	where bar.open_pr is not null
            	and bar.close_pr is not null
            	and bar.dlyhigh is not null
            	and bar.dlylow is not null
            	and bar.dlynumtrd is not null
            )

            select a.ticker, 
                   a.current_date, 
                   a.before_date, 
                   a.after_date,
                   (a.before_avg_dlynumtrd + a.after_avg_dlynumtrd)/2.0 as avg_dlynumtrd,
                   a.dlynumtrd
            from anom_details a
    """, conn)

inter_anom_df = pd.read_sql("""
    select eir.ticker, eir.current_date 
    from ev_interday_results eir 
    where avg_dlynumtrd is not null
    and high_extreme is not null 
    and low_extreme is not null
    order by current_date
    """, conn)

**FIXME**: Change this date range to 100 days before and after and choose an extreme value period of 5 days (approx. one business week)
- From here, we should double check that all of our identified anomalies actually classify as such. As long as we have a total sample size of >70 observations, we should be good to go.

In [8]:
# let's restrict this set...
inter_anom_df = inter_anom_df.drop_duplicates(subset=["ticker", "current_date"])

In [9]:
# faster method: get unique current_dates and create a dataframe
current_dt_lst = list(inter_anom_df['current_date'].unique())
before_dt_lst = [mkt_utils.get_before_date(dt, 100) for dt in current_dt_lst]
after_dt_lst = [mkt_utils.get_after_date(dt, 100) for dt in current_dt_lst]

In [10]:
dates_df = pd.DataFrame({"current_date": current_dt_lst, "before_date": before_dt_lst, "after_date": after_dt_lst})

In [11]:
# get 100 days before for each symbol
inter_anom_df = inter_anom_df.merge(dates_df, how = "left", left_on="current_date", right_on="current_date")

In [12]:
run_already_processed = True

In [13]:
if run_already_processed:
    already_processed = pd.read_csv("ev_interday_results.csv")
    already_processed = already_processed.drop_duplicates()
    merged_df = inter_anom_df.merge(already_processed[['ticker', 'current_date']], on=['ticker', 'current_date'], how='left', indicator=True)
    inter_filtered_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])
else:
    inter_filtered_df = inter_anom_df.copy()

In [ ]:
if run_inter_pdata:
    # chunk the DataFrame into 100-row chunks and process each chunk
    chunk_size = 100
    output_file = "ev_interday_results.csv"
    
    for start in tqdm(range(0, len(inter_filtered_df), chunk_size)):
        chunk = inter_filtered_df[start:start + chunk_size]
        processed_chunk = ev.process_interday_data(chunk)
        
        # write to CSV
        if start == 0 and not run_already_processed:
            # Write the header in the first iteration
            processed_chunk.to_csv(output_file, mode='w', index=False)
        else: 
            processed_chunk.to_csv(output_file, mode='a', index=False, header=False)

/opt/anaconda3/lib/python3.8/site-packages/pyextremes/extremes/block_maxima.py:138: NoDataBlockWarning: 4 blocks contained no data                             | 0/95 [00:00<?, ?it/s]
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/pyextremes/extremes/block_maxima.py:138: NoDataBlockWarning: 4 blocks contained no data
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/pyextremes/extremes/block_maxima.py:138: NoDataBlockWarning: 4 blocks contained no data
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/pyextremes/extremes/block_maxima.py:138: NoDataBlockWarning: 4 blocks contained no data
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/pyextremes/extremes/block_maxima.py:138: NoDataBlockWarning: 4 blocks contained no data
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/pyextremes/extremes/block_maxima.py:138: NoDataBlockWarning: 3 blocks contained no data
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/pyextremes/extrem

## Exploring Results

In this section, we'll explore our extreme value-scored results collected above.

In [ ]:
pd.read_sql("""
    with before_aft as (
    	select bar.*
    	from before_after_results bar
    	where bar.open_pr is not null
    	and bar.close_pr is not null
    	and bar.dlyhigh is not null
    	and bar.dlylow is not null
    	and bar.dlynumtrd is not null
    ),
    
    ev_vals as (
    	select *
    	from ev_intraday_results er 
    	where high_score is not null
    	and low_score is not null
    )
    
    select b.*, e.high_extreme, e.low_extreme, (e.high_score * 100) as high_score, (e.low_score * 100) as low_score
    from before_aft b
    join ev_vals e
    on (b.ticker, b.current_date) = (e.ticker, e.current_date)
    order by e.high_score
""", conn)

In [ ]:
mkt_utils.multiday_chart("ALZN", "2021-09-30", high = True, diff_num = 15)

In [ ]:
mkt_utils.multiday_chart("MNPR", "2020-06-16", high = True, diff_num = 30)

In [ ]:
mnpr_trades = ev.get_daily_trades("2020-06-16", "2020-06-10", "2020-06-24", "MNPR")

In [ ]:
mnpr_trades.volume.plot()

In [ ]:
mnpr_trades.sort_values("avg_price_diff", ascending=False)

In [ ]:
mnpr_trades.avg_price_diff.plot()

In [ ]:
test = pd.read_csv("processed_data.csv")

In [ ]:
test.drop_duplicates().reset_index(drop=True).to_csv("processed_data.csv", index=False)

## Quality Checks

In [ ]:
err_obs = pd.read_sql("""with before_aft as (
                            select bar.*
                            from before_after_results bar
                            where bar.open_pr is not null
                            and bar.close_pr is not null
                            and bar.dlyhigh is not null
                            and bar.dlylow is not null
                            and bar.dlynumtrd is not null
                        ),
                        
                        ev_vals as (
                            select *
                            from ev_results er 
                            where high_score is not null
                            and low_score is not null
                        ),
                        
                        intraday_anomalies as (
                            select ha.ticker, 
                                   ha.current_date, 
                                   ha.before_date, 
                                   ha.after_date, 
                                   ha.vix_quantile, 
                                   ha.vlty_quantile,
                                   ha.before_pct_diff, 
                                   ha.after_pct_diff,
                                   ha.anomaly_fl 
                            from high_anomalies ha 
                            union
                            select la.ticker, 
                                   la.current_date, 
                                   la.before_date, 
                                   la.after_date, 
                                   la.vix_quantile, 
                                   la.vlty_quantile,
                                   la.before_pct_diff, 
                                   la.after_pct_diff,
                                   la.anomaly_fl 
                            from low_anomalies la 
                        ),
                        
                        ev_scores as (
                            select b.*, 
                                   e.high_extreme, 
                                   e.low_extreme, 
                                   (e.high_score * 100) as high_score, 
                                   (e.low_score * 100) as low_score,
                                   i.before_pct_diff,
                                   i.after_pct_diff,
                                   i.anomaly_fl,
                                   i.vlty_quantile
                            from before_aft b
                            join ev_vals e
                            on (b.ticker, b.current_date) = (e.ticker, e.current_date)
                            left join intraday_anomalies i
                            on (b.ticker, b.current_date) = (i.ticker, i.current_date)
                            order by e.high_score
                        )
                        
                        select *
                        from ev_scores
                        where high_extreme > dlyhigh or abs(low_extreme) > dlyhigh
                    """, conn)

In [ ]:
err_obs = err_obs.drop_duplicates(["ticker", "current_date"]).reset_index(drop=True)

In [ ]:
err_obs

In [ ]:
daily_tr = db.raw_sql("""
    select *,
           date_trunc('second', time_m) as trunc_time
    from taqm_2014.ctm_2014 c 
    where c.date = date('2014-01-22')
    and c.sym_root = 'VRNG'
    and time_m >= '09:30:00'
    and time_m <= '16:00:00'
    and tr_scond !~ '[OPQ65]'
    and tr_corr = '00' -- non-corrected/cancelled trades
    and sym_suffix is null -- exclude warrants, rights, units, etc
""")

In [ ]:
daily_tr[daily_tr.time_m > dt.time(12, 39, 20, 0)]

In [ ]:
daily_tr[(daily_tr.index > 35) & (daily_tr.index < 60)]

In [ ]:
daily_tr.avg_price.plot()

In [ ]:
daily_tr.num_trade_diff.plot()

## Examining Poor Fits

In [ ]:
unfit_ev = pd.read_sql( """
    with fitted_evs as (
    	select *
    	from ev_intraday_results et
    	where et.high_extreme is not null
    	and et.low_extreme is not null 
    	and et.high_score is not not null 
    	and et.low_score is not null
    ),
    
    unfitted as (
    	select *
    	from fitted_evs
    	where high_pvalue < 0.05
    )
    
    select u.*,
           bar.before_avg_dlynumtrd,
           bar.after_avg_dlynumtrd,
           bar.dlynumtrd
    from unfitted u
    left join before_after_results bar
    on (u.ticker, u.current_date) = (bar.ticker, bar.current_date)
    order by u.high_score asc, bar.before_avg_dlynumtrd desc
    """, conn)

In [ ]:
unfit_ev

Using the K-S test statistics, let's determine which EV distributions provide a poor fit.

In [ ]:
unfit_ev[["block_size"]].value_counts()

The majority of poorly fit EV distributions are at the 1H frequency.

In [ ]:
unfit_ev.sort_values("avg_dlynumtrd", ascending=False)

Interday extreme values...

In [ ]:
htoo_test = mkt_utils.intraday_df_w_dates(symbol="HTOO", before_dt="2020-11-18", after_dt = "2021-01-04")

In [ ]:
htoo_test[:50]

In [ ]:
moxc_test.volume.plot()

In [ ]:
moxc_test.qtr.plot()

In [ ]:
mkt_utils.multiday_chart("MOXC", "2020-04-08", diff_num=15)

In [ ]:
mkt_utils.multiday_candlestick("MOXC", "2020-04-08", diff_num=30)